In [1]:
# Author: Antoine DELPLACE
# Last update: 17/01/2020
"""
Perform a statistic analysis of the Neural network classifier.

Parameters
----------
data_window_botnetx.h5         : extracted data from preprocessing1.py
data_window3_botnetx.h5        : extracted data from preprocessing2.py
data_window_botnetx_labels.npy : label numpy array from preprocessing1.py
nb_prediction                  : number of predictions to perform

Return
----------
Print train and test mean accuracy, precison, recall, f1
"""

'\nPerform a statistic analysis of the Neural network classifier.\n\nParameters\n----------\ndata_window_botnetx.h5         : extracted data from preprocessing1.py\ndata_window3_botnetx.h5        : extracted data from preprocessing2.py\ndata_window_botnetx_labels.npy : label numpy array from preprocessing1.py\nnb_prediction                  : number of predictions to perform\n\nReturn\n----------\nPrint train and test mean accuracy, precison, recall, f1\n'

In [2]:
import numpy as np
import pandas as pd
from scipy.sparse import csc_matrix
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import h5py
import time

In [3]:
! python3 -m pip install --upgrade keras
! python3 -m pip install --upgrade tensorflow

Defaulting to user installation because normal site-packages is not writeable
  Using cached keras-2.8.0-py2.py3-none-any.whl (1.4 MB)
  Attempting uninstall: keras
    Found existing installation: keras 2.6.0
    Uninstalling keras-2.6.0:
      Successfully uninstalled keras-2.6.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.6.2 requires keras<2.7,>=2.6.0, but you have keras 2.8.0 which is incompatible.
Defaulting to user installation because normal site-packages is not writeable
  Using cached keras-2.6.0-py2.py3-none-any.whl (1.3 MB)
  Attempting uninstall: keras
    Found existing installation: keras 2.8.0
    Uninstalling keras-2.8.0:
      Successfully uninstalled keras-2.8.0


In [4]:
import tensorflow as tf
import tensorflow.keras.backend as K
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.utils import to_categorical, plot_model
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Input, Conv2D, BatchNormalization, Activation, AveragePooling2D, Dense, Dropout, Flatten, Lambda, MaxPool2D, Conv2DTranspose, UpSampling2D, Concatenate, Add
from tensorflow.keras import regularizers, optimizers
from keras.preprocessing import image
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

In [5]:
from sklearn import model_selection, feature_selection, utils, ensemble, linear_model, metrics

In [6]:
print("Import data")

Import data


In [7]:
X = pd.read_hdf('data_window_botnet3.h5', key='data')
X.reset_index(drop=True, inplace=True)

In [8]:
X2 = pd.read_hdf('data_window3_botnet3.h5', key='data')
X2.reset_index(drop=True, inplace=True)

In [9]:
X = X.join(X2)

In [10]:
X.drop('window_id', axis=1, inplace=True)

In [11]:
y = X['Label_<lambda>']
X.drop('Label_<lambda>', axis=1, inplace=True)

In [12]:
labels = np.load("data_window_botnet3_labels.npy", allow_pickle=True)

In [13]:
print(X.columns.values)
print(labels)
print(np.where(labels == 'flow=From-Botne')[0][0])

['counts' 'Sport_nunique' 'DstAddr_nunique' 'Dport_nunique' 'Dur_sum'
 'Dur_mean' 'Dur_std' 'Dur_max' 'Dur_median' 'TotBytes_sum'
 'TotBytes_mean' 'TotBytes_std' 'TotBytes_max' 'TotBytes_median'
 'SrcBytes_sum' 'SrcBytes_mean' 'SrcBytes_std' 'SrcBytes_max'
 'SrcBytes_median' 'Sport_RU' 'DstAddr_RU' 'Dport_RU']
['flow=Background' 'flow=To-Backgro' 'flow=From-Backg' 'flow=From-Norma'
 'flow=To-Normal-' 'flow=Normal-V42' 'flow=From-Botne']
6


In [14]:
y_bin6 = y==np.where(labels == 'flow=From-Botne')[0][0]
print("y", np.unique(y, return_counts=True))

y (array([list([0]), list([1]), list([2]), list([3]), list([4]), list([6])],
      dtype=object), array([2207092,   18047,     263,     984,      48,     286]))


In [15]:
## NN
filename_weights = "model.h5"

In [16]:
def fprecision(y_true, y_pred):	
    """Precision metric.	
    Only computes a batch-wise average of precision. Computes the precision, a
    metric for multi-label classification of how many selected items are
    relevant.
    """	
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))	
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))	
    precision = true_positives / (predicted_positives + K.epsilon())	
    return precision

In [17]:
def frecall(y_true, y_pred):	
    """Recall metric.	
    Only computes a batch-wise average of recall. Computes the recall, a metric
    for multi-label classification of how many relevant items are selected.	
    """	
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))	
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))	
    recall = true_positives / (possible_positives + K.epsilon())	
    return recall

In [18]:
def ff1_score(y_true, y_pred):
    """Computes the F1 Score
    Only computes a batch-wise average of recall. Computes the recall, a metric
    for multi-label classification of how many relevant items are selected.	
    """
    p = fprecision(y_true, y_pred)
    r = frecall(y_true, y_pred)
    return (2 * p * r) / (p + r + K.epsilon())

In [19]:
def get_model(inputs, dropout=0.5, batchnorm=True):
    x = Dense(256, input_shape=(22,))(inputs)
    if batchnorm:
        x = BatchNormalization()(x)
    x = Activation("relu")(x)
    x = Dropout(dropout)(x)

    x = Dense(128, input_shape=(256,))(x)
    if batchnorm:
        x = BatchNormalization()(x)
    x = Activation("relu")(x)
    x = Dropout(dropout)(x)

    x = Dense(1, input_shape=(128,))(x)
    outputs = Activation("sigmoid")(x)
    
    model = Model(inputs=[inputs], outputs=[outputs])
    return model

In [20]:
nb_prediction = 3
np.random.seed(seed=123456)
tab_seed = np.random.randint(0, 1000000000, nb_prediction)
print(tab_seed)

[545331265  64051946 930796018]


In [21]:
tab_train_precision = np.array([0.]*nb_prediction)
tab_train_recall = np.array([0.]*nb_prediction)
tab_train_fbeta_score = np.array([0.]*nb_prediction)

In [22]:
tab_test_precision = np.array([0.]*nb_prediction)
tab_test_recall = np.array([0.]*nb_prediction)
tab_test_fbeta_score = np.array([0.]*nb_prediction)

In [23]:
for i in range(0, nb_prediction):
    X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y_bin6, test_size=0.33, random_state=tab_seed[i])

    print(i)
    print("y_train", np.unique(y_train, return_counts=True))
    print("y_test", np.unique(y_test, return_counts=True))

    inputs = Input((22,), name='input')
    model = get_model(inputs, dropout=0, batchnorm=1)

    callbacks = [
        ModelCheckpoint(filename_weights, verbose=1, save_best_only=True, save_weights_only=True)
    ]

    model.compile(optimizer=optimizers.Adam(lr=1e-3), loss=["binary_crossentropy"], metrics=[fprecision, frecall, ff1_score])
    #model.summary()

    tps = time.time()
    results = model.fit(X_train, y_train, batch_size=32, epochs=20, validation_split=0.15, shuffle=True, class_weight=None, verbose=0, callbacks=callbacks)
    print("Execution time = ", time.time()-tps)

    model.load_weights(filename_weights)

    y_pred_train = model.predict(X_train, batch_size=32, verbose=0)
    y_pred_train_bin = (y_pred_train > 0.5).astype(np.uint8)
    precision, recall, fbeta_score, support = metrics.precision_recall_fscore_support(y_train, y_pred_train_bin)
    tab_train_precision[i] = precision[1]
    tab_train_recall[i] = recall[1]
    tab_train_fbeta_score[i] = fbeta_score[1]

    y_pred_test = model.predict(X_test, batch_size=32, verbose=0)
    y_pred_test_bin = (y_pred_test > 0.5).astype(np.uint8)
    precision, recall, fbeta_score, support = metrics.precision_recall_fscore_support(y_test, y_pred_test_bin)
    tab_test_precision[i] = precision[1]
    tab_test_recall[i] = recall[1]
    tab_test_fbeta_score[i] = fbeta_score[1]

0
y_train (array([False,  True]), array([1491711,     191]))
y_test (array([False,  True]), array([734723,     95]))


/home/2019a7ps1343h/.local/lib/python3.6/site-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")



Epoch 00001: val_loss improved from inf to 0.00073, saving model to model.h5

Epoch 00002: val_loss did not improve from 0.00073

Epoch 00003: val_loss improved from 0.00073 to 0.00069, saving model to model.h5

Epoch 00004: val_loss improved from 0.00069 to 0.00062, saving model to model.h5

Epoch 00005: val_loss improved from 0.00062 to 0.00055, saving model to model.h5

Epoch 00006: val_loss improved from 0.00055 to 0.00050, saving model to model.h5

Epoch 00007: val_loss did not improve from 0.00050

Epoch 00008: val_loss did not improve from 0.00050

Epoch 00009: val_loss did not improve from 0.00050

Epoch 00010: val_loss did not improve from 0.00050

Epoch 00011: val_loss improved from 0.00050 to 0.00043, saving model to model.h5

Epoch 00012: val_loss did not improve from 0.00043

Epoch 00013: val_loss did not improve from 0.00043

Epoch 00014: val_loss did not improve from 0.00043

Epoch 00015: val_loss did not improve from 0.00043

Epoch 00016: val_loss improved from 0.00043

/home/2019a7ps1343h/.local/lib/python3.6/site-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")



Epoch 00001: val_loss improved from inf to 0.00055, saving model to model.h5

Epoch 00002: val_loss did not improve from 0.00055

Epoch 00003: val_loss did not improve from 0.00055

Epoch 00004: val_loss did not improve from 0.00055

Epoch 00005: val_loss did not improve from 0.00055

Epoch 00006: val_loss improved from 0.00055 to 0.00042, saving model to model.h5

Epoch 00007: val_loss improved from 0.00042 to 0.00035, saving model to model.h5

Epoch 00008: val_loss improved from 0.00035 to 0.00019, saving model to model.h5

Epoch 00009: val_loss did not improve from 0.00019

Epoch 00010: val_loss did not improve from 0.00019

Epoch 00011: val_loss did not improve from 0.00019

Epoch 00012: val_loss did not improve from 0.00019

Epoch 00013: val_loss did not improve from 0.00019

Epoch 00014: val_loss did not improve from 0.00019

Epoch 00015: val_loss did not improve from 0.00019

Epoch 00016: val_loss did not improve from 0.00019

Epoch 00017: val_loss did not improve from 0.00019


/home/2019a7ps1343h/.local/lib/python3.6/site-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")



Epoch 00001: val_loss improved from inf to 0.00234, saving model to model.h5

Epoch 00002: val_loss did not improve from 0.00234

Epoch 00003: val_loss did not improve from 0.00234

Epoch 00004: val_loss improved from 0.00234 to 0.00210, saving model to model.h5

Epoch 00005: val_loss improved from 0.00210 to 0.00191, saving model to model.h5

Epoch 00006: val_loss improved from 0.00191 to 0.00059, saving model to model.h5

Epoch 00007: val_loss did not improve from 0.00059

Epoch 00008: val_loss did not improve from 0.00059

Epoch 00009: val_loss did not improve from 0.00059

Epoch 00010: val_loss did not improve from 0.00059

Epoch 00011: val_loss did not improve from 0.00059

Epoch 00012: val_loss did not improve from 0.00059

Epoch 00013: val_loss did not improve from 0.00059

Epoch 00014: val_loss did not improve from 0.00059

Epoch 00015: val_loss did not improve from 0.00059

Epoch 00016: val_loss did not improve from 0.00059

Epoch 00017: val_loss did not improve from 0.00059


/home/2019a7ps1343h/.local/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/2019a7ps1343h/.local/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [24]:
print("Train")
print("precision = ", tab_train_precision.mean(), tab_train_precision.std(), tab_train_precision)
print("recall = ", tab_train_recall.mean(), tab_train_recall.std(), tab_train_recall)
print("fbeta_score = ", tab_train_fbeta_score.mean(), tab_train_fbeta_score.std(), tab_train_fbeta_score)

Train
precision =  0.653558052434457 0.46241411916332475 [1.         0.96067416 0.        ]
recall =  0.5802887168980955 0.41313613097393037 [0.81151832 0.92934783 0.        ]
fbeta_score =  0.613568379480301 0.4343154915115127 [0.89595376 0.94475138 0.        ]


In [25]:
print("Test")
print("precision = ", tab_test_precision.mean(), tab_test_precision.std(), tab_test_precision)
print("recall = ", tab_test_recall.mean(), tab_test_recall.std(), tab_test_recall)
print("fbeta_score = ", tab_test_fbeta_score.mean(), tab_test_fbeta_score.std(), tab_test_fbeta_score)

Test
precision =  0.6564625850340136 0.4643573491371904 [1.         0.96938776 0.        ]
recall =  0.5841417268661852 0.41549872751163763 [0.82105263 0.93137255 0.        ]
fbeta_score =  0.6172447013487475 0.43690247972545937 [0.9017341 0.95      0.       ]
